In [1]:
import requests
import pickle as pkl
from time import sleep
from tqdm import tqdm
import pandas as pd 
from bs4 import BeautifulSoup
import cloudscraper 

def parse(html):
    """Parses the HTML for the price of the product."""
    soup = BeautifulSoup(html, 'html.parser')
    try:
        # Find the span element with the class "product-price"
        output = soup.find('span', class_='common__EiReviewDetailsStyle__newUiJobLine').text
        return output
    except:
        print("Error parsing for location")

In [13]:
API_TOKEN = input("Enter your API token: ")

In [32]:
dataset = []

In [ ]:
url = "https://wextractor.com/api/v1/reviews/glassdoor"

##Specify page numbers
offset_range = [i * 10 for i in range(250, 350)] # post ids

for offset in tqdm(offset_range, desc="Downloading data"):
    params = {
        "id": "575263",
        "auth_token": API_TOKEN,
        "language": "en",
        "offset": offset,
    }

    # Make the API request
    response = requests.get(url, params=params)

    # Process the response
    if response.status_code == 200:
        data = response.json()
        ### Add to the list
        dataset.extend(data["reviews"])

    else:
        print("Error:", response.status_code)


In [3]:
try: 
    #with open("glassdoor_2500_3500.pkl", "wb") as f:
    #   pkl.dump(dataset, f)
    df = pd.DataFrame(dataset).set_index("id")
    df["datetime"] = pd.to_datetime(df["datetime"])
    df.to_csv("glassdoor_2500_3500.csv")
except:
    df = pd.read_csv("glassdoor_2500_3500.csv", index_col="id")
    df["raw_meta"] = ""

In [14]:
# Create cloudscraper instance 
scraper = cloudscraper.create_scraper() 
# Start scrapping
for i, row in tqdm(df.iterrows(), desc="Scraping locations"):
    ###### in case of error #################
    # aka if we already have the data -> skip
    if row["raw_meta"] != "":
        continue
    #########################################
    page = scraper.get(df.loc[i, "url"])
    loc = parse(page.content)
    print(loc)
    df.loc[i, "raw_meta"] = loc
# Or: scraper = cloudscraper.CloudScraper() # CloudScraper inherits from requests.Session 


Scraping locations: 954it [11:08,  1.87s/it]

Feb 20, 2022 - Coordinator in Warsaw, Masovia


Scraping locations: 955it [11:10,  1.73s/it]

Feb 20, 2022 - Team Leader in Angeles, Pampanga


Scraping locations: 956it [11:12,  1.80s/it]

...

Feb 20, 2022 - Regional Head of Operations in Amsterdam


Scraping locations: 959it [11:16,  1.55s/it]

Feb 20, 2022 - Delivery Driver 


Scraping locations: 960it [11:18,  1.42it/s]

Feb 20, 2022 - Delivery Driver 


In [27]:
def get_location(x):
    try:
        if "in " in x:
            return x.split("in")[-1].strip()
        else: 
            return None
    except:
        return None
df["location"] = df.apply(lambda x: get_location(x["raw_meta"]), axis=1)
df.to_csv("glassdoor_2500_3500_with_loc.csv")

id
66362518               None
66359165               None
66344375    Los Angeles, CA
66335091       New York, NY
66329753               None
                 ...       
60008846             Sydney
60005224        Phoenix, AZ
59999562          Amsterdam
59999533               None
59996398               None
Name: location, Length: 960, dtype: object